In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sn
import time
import pickle
%matplotlib inline
plt.style.use('seaborn')

In [13]:
%time
df = pd.read_csv('Taxi_Trips_NearSouth_Updated.csv')

CPU times: user 4 µs, sys: 7 µs, total: 11 µs
Wall time: 35 µs


In [14]:
df.shape

(2204222, 18)

In [17]:
df.head(5)

,Trip Start Timestamp,Trip Seconds,Trip Miles,Pickup Community Area,Fare,Tips,Tolls,Extras,Trip Total,Pickup Centroid Latitude,Pickup Centroid Longitude,Dropoff Centroid Latitude,Dropoff Centroid Longitude
0,01/01/2013 12:00:00 AM,360.0,2.3,33,7.85,0.0,0.0,2.0,9.85,41.859350,-87.617358,41.884987,-87.620993
1,01/01/2013 12:00:00 AM,480.0,0.0,33,7.65,1.0,0.0,1.0,9.65,41.859350,-87.617358,41.880994,-87.632746
2,01/01/2013 12:00:00 AM,1200.0,3.7,33,13.45,0.0,0.0,1.5,14.95,41.859350,-87.617358,41.898332,-87.620763
3,01/01/2013 12:00:00 AM,60.0,0.3,33,3.65,0.0,0.0,1.0,4.65,41.859350,-87.617358,41.859350,-87.617358
4,01/01/2013 12:00:00 AM,900.0,0.0,33,10.65,0.0,0.0,2.0,12.65,41.857184,-87.620335,41.899602,-87.633308


In [16]:
df = df.drop(['Trip ID','Taxi ID','Trip End Timestamp','Dropoff Community Area','Payment Type'],axis=1)

In [18]:
%time
# Change datetime format for trip start and remove original timestamp 
df['Trip_Start'] = pd.to_datetime(df['Trip Start Timestamp'])
df.drop('Trip Start Timestamp',axis=1, inplace=True)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.91 µs


In [19]:
dfn = df.copy()

In [20]:
dfn['month'] = dfn.Trip_Start.dt.month
dfn['time'] = dfn.Trip_Start.dt.time
dfn['day']=dfn.Trip_Start.dt.dayofweek
dfn = dfn.drop('Trip_Start', axis=1)

In [28]:
dfn = dfn.drop(['Pickup Centroid Latitude','Pickup Centroid Longitude','Dropoff Centroid Latitude','Dropoff Centroid Longitude'],axis=1)

In [36]:
dfn.drop('Pickup Community Area', axis=1, inplace=True)

In [37]:
dfn.head(5)

,Trip Seconds,Trip Miles,Fare,Tips,Tolls,Extras,Trip Total,month,time,day,season,rate
0,360.0,2.3,7.85,0.0,0.0,2.0,9.85,1,00:00:00,1,Winter,0.027361
1,480.0,0.0,7.65,1.0,0.0,1.0,9.65,1,00:00:00,1,Winter,0.020104
2,1200.0,3.7,13.45,0.0,0.0,1.5,14.95,1,00:00:00,1,Winter,0.012458
3,60.0,0.3,3.65,0.0,0.0,1.0,4.65,1,00:00:00,1,Winter,0.077500
4,900.0,0.0,10.65,0.0,0.0,2.0,12.65,1,00:00:00,1,Winter,0.014056


In [24]:
# create a season column
def f(row):
    if row['month'] in (12, 1, 2):
        val = 'Winter'
    elif row['month'] in [3,4,5]:
        val = 'Spring'
    elif row['month'] in [6,7,8]:
        val = 'Summer'
    else:
        val = 'Fall'
    return val

dfn['season'] = dfn.apply(f, axis=1)

In [31]:
dfn['rate']=dfn['Trip Total']/dfn['Trip Seconds']

In [39]:
gcol = ['day','month','time','season']
df_analysis = dfn.groupby(gcol).aggregate({
    'Trip Miles':'count',
    'Trip Total':'mean',
    'Trip Seconds': 'mean',
    'rate': 'mean'
}).reset_index()

In [45]:
df_analysis.tail(5)

,day,month,time,season,Count_Trips,Trip Total,Trip Seconds,rate,num_minutes
8059,6,12,22:45:00,Winter,65,10.610769,560.307692,inf,1365
8060,6,12,23:00:00,Winter,59,11.468644,587.586207,inf,1380
8061,6,12,23:15:00,Winter,74,11.552838,638.918919,inf,1395
8062,6,12,23:30:00,Winter,71,12.510845,638.571429,inf,1410
8063,6,12,23:45:00,Winter,61,11.543115,626.918033,0.022561,1425


In [42]:
df_analysis.rename(columns={'Trip Miles':'Count_Trips'},inplace=True)

In [44]:
df_analysis['num_minutes'] = df_analysis['time'].transform(lambda dt: dt.hour*60 + dt.minute)